### Imports

In [1]:
import numpy as np
from numpy.polynomial.hermite import hermval as H

from scipy.special import factorial as fac
from scipy.special import eval_hermite

from sympy import symbols, diff, exp, Poly

from mpmath import mp

from numba import jit, njit


### Functions for creating the coefficient table

In [2]:
def hermite_sympy(n):

    x = symbols("x")
    if n == 0:
        return 1
    else:
        return ((-1)**n) * exp(x**2) * diff(exp(-x**2), x, n)

def create_hermite_coefficients_table(N,integer):

  x = symbols("x")
  C = np.zeros((N+1,N+1),dtype = object) if(integer) else np.zeros((N+1,N+1),dtype = np.float64)
  C[0,N] = 1

  for n in range(1,N+1):
    c = Poly(hermite_sympy(n),x).all_coeffs()
    for index in range(n,-1,-1):
      C[n,(N+1)-index-1] = c[n-index] if(integer) else float(c[n-index])

  return C

### $\psi_{n}(x)$ with numpy

In [3]:
@jit(forceobj=True,looplift=True, boundscheck=False)
def psiFock_numpy(n,x): # baseline-2 to infinit precision
  n_array = np.zeros(n+1)
  n_array[n] = 1
  return (2**(-0.5*n))*(fac(n)**(-0.5))*(np.pi**(-0.25))*np.exp(-(x**2)/2)*H(x, n_array)

### $\psi_{n}(x)$ with scipy

In [4]:
@jit(forceobj=True,looplift=True, boundscheck=False)
def psiFock_scipy(n,x):# baseline-3 to infinit precision
  return (2**(-0.5*n))*(fac(n)**(-0.5))*(np.pi**(-0.25))*np.exp(-(x**2)/2)*eval_hermite(n,x)

### $\psi_{n}(x)$ with the Table of Coefficients and Numba optimization

In [8]:
C_float = create_hermite_coefficients_table(100,False)

@njit(nopython=True,looplift=True, nogil=True, boundscheck=False)
def factorial_opt(n):
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

@njit(nopython=True,looplift=True,nogil=True, boundscheck=False)
def psiFock_C_table_opt(n,x):

  coeff_vector = C_float[n]
  c_size = C_float.shape[0]
  zero_index_vector = np.where(coeff_vector == 0)[0]
  x_power = np.array([ 0.0 if(i in zero_index_vector) else x**(c_size-i-1) for i in range(c_size)],dtype=np.float64)
  return np.dot(coeff_vector, x_power)*np.exp(-(x**2)/2)*(2**(-0.5*n))*(factorial_opt(n)**(-0.5))*(np.pi**(-0.25))


### Hibrid $\psi_{n}(x)$

In [9]:
def psiFock_C_table_opt_scipy(n,x):
   
   if(n<=20):
      return psiFock_C_table_opt(n,x)
   else:
      return psiFock_scipy(n,x)